In [1]:
import json
import pandas as pd
import time

In [2]:
def get_relation_label(rel_id):
    label = ''
    API_ENDPOINT = "https://www.wikidata.org/w/api.php"
    params = {
        'action': 'wbgetentities',
        'format': 'json',
        'languages': 'en',
        'props': 'labels',
        'ids': ''
    }
      
    params['ids'] = str(rel_id)

    try:
        response = requests.get(API_ENDPOINT, params = params).json()['entities']
        label = response[str(rel_id)]['labels']['en']['value']
    except:
        return (rel_id, label)
    return (rel_id, label)

In [29]:
import requests
import time

def find_relation_names_and_ids(qid):
    relation_name_id = dict()

    url = 'https://query.wikidata.org/sparql'
    query = '''
    SELECT Distinct ?wd ?wdLabel
    WHERE {
      VALUES (?s) {(wd:entity)}
      ?s ?wdt ?o .
      ?wd wikibase:directClaim ?wdt .
      ?wd rdfs:label ?wdLabel .
      OPTIONAL {
        ?o rdfs:label ?oLabel .
        FILTER (lang(?oLabel) = "en")
      }
      FILTER (lang(?wdLabel) = "en")
      BIND (COALESCE(?oLabel, ?o) AS ?ooLabel)
     }
    '''
    copy_query = query.replace('entity', qid)
#     time.sleep(3)
    r = requests.get(url, params = {'format': 'json', 'query': copy_query})
    try:
        data = r.json()
        for rel in data['results']['bindings']:
#             if len(rel['wdLabel']['value']) < 35:
            relation_name_id[rel['wdLabel']['value']] = rel['wd']['value'].rsplit('/', 1)[1]
        return relation_name_id
    except:
        print(r)
        return ''

In [4]:
# jupyter notebook never finishes processing using multiprocessing python 3
# https://stackoverflow.com/questions/47313732/jupyter-notebook-never-finishes-processing-using-multiprocessing-python-3

from multiprocessing import Pool
from functools import partial
import inspect
import time
import os
import requests


def parallal_task(func, iterable, *params): 
    with open(f'./tmp_func.py', 'w') as file:
        file.write("import requests \n")
        file.write(inspect.getsource(func).replace(func.__name__, 'task'))

    from tmp_func import task
    pool = Pool(processes=15)
    res = pool.map(task, iterable)
    pool.close()
    
    os.remove('./tmp_func.py')
    return res

In [5]:
from multiprocessing import Pool
from functools import partial
import inspect
import time
import os
import requests


def parallal_task_2(func, iterable, *params): 
    with open(f'./tmp_func_2.py', 'w') as file:
        file.write("import requests \n")
        file.write("import time \n")
        file.write(inspect.getsource(func).replace(func.__name__, 'task'))

    from tmp_func_2 import task
    pool = Pool(processes=15)
    res = pool.map(task, iterable)
    pool.close()
    
    os.remove('./tmp_func_2.py')
    return res

In [6]:
with open('2-NEL_Data/0-raw_data/simple_questions_v2/annotated_wd_data_train.txt', encoding='utf-8') as file:
    lines = file.readlines()

# print(len(lines))
training_data = []
for line in lines:
    line = line.strip().split('\t', 3)
    del line[2]
    
    training_data.append(line)
    
qids = [line[0] for line in training_data]
target_relation = [line[1].replace('R', 'P') for line in training_data]
questions = [line[2] for line in training_data]

In [7]:
qids[0]

'Q126399'

In [59]:
def get_entity_relation_ids(ent_id):
    relations = []
    API_ENDPOINT = "https://www.wikidata.org/w/api.php"
    params = {
        'action': 'wbgetentities',
        'format': 'json',
        'languages': 'en',
        'props': 'claims',
        'ids': ''
    }
        
    params['ids'] = str(ent_id)
    try:
        response = requests.get(API_ENDPOINT, params = params).json()['entities']
        print(response)
        relations = list(response[str(ent_id)]['claims'].keys())
    except:
        return (ent_id, relations)
    return (ent_id, relations)

In [60]:
(_, claims) = get_entity_relation_ids('Q126399')

{'Q126399': {'type': 'item', 'id': 'Q126399', 'claims': {'P373': [{'mainsnak': {'snaktype': 'value', 'property': 'P373', 'hash': 'a628796d86bec5f43a27156210ee152fbee3cec1', 'datavalue': {'value': 'Warner Bros. Entertainment', 'type': 'string'}, 'datatype': 'string'}, 'type': 'statement', 'id': 'q126399$3B2A1FB8-16B9-4D74-BB8B-3823770A99F6', 'rank': 'normal'}], 'P31': [{'mainsnak': {'snaktype': 'value', 'property': 'P31', 'hash': '03f36c5d44daa15d8ad8a4f337bd7cae11b4d847', 'datavalue': {'value': {'entity-type': 'item', 'numeric-id': 1107679, 'id': 'Q1107679'}, 'type': 'wikibase-entityid'}, 'datatype': 'wikibase-item'}, 'type': 'statement', 'id': 'Q126399$EAF40450-3FCB-478E-98B4-5C2BD6CED344', 'rank': 'normal'}, {'mainsnak': {'snaktype': 'value', 'property': 'P31', 'hash': '09cda8dc5df05f496773981eeb13389efcccf3d1', 'datavalue': {'value': {'entity-type': 'item', 'numeric-id': 1762059, 'id': 'Q1762059'}, 'type': 'wikibase-entityid'}, 'datatype': 'wikibase-item'}, 'type': 'statement', 'id'

In [47]:
len(claims)

93

In [48]:
ids = list(find_relation_names_and_ids('Q126399').values())

In [49]:
for id in claims:
    if id not in ids:
        print(id)
        

P345
P691
P8317


In [50]:
for id in ids:
    if id not in claims:
        print('fuck')

In [38]:
ids

['P495',
 'P364',
 'P136',
 'P31',
 'P3402',
 'P272',
 'P161',
 'P437',
 'P462',
 'P57',
 'P9586',
 'P1476',
 'P646',
 'P2346',
 'P7334',
 'P2047',
 'P905',
 'P2529',
 'P3107',
 'P577',
 'P2603',
 'P1562',
 'P3302',
 'P345',
 'P1970',
 'P2638',
 'P3145',
 'P4529',
 'P5786',
 'P6127',
 'P480',
 'P3143',
 'P3138',
 'P3844',
 'P4947',
 'P6398',
 'P7970']

In [15]:
counter = 0
for qid in qids[:100]:
#     print(qid)
    counter += 1
    print(counter)
    if not find_relation_names_and_ids(str(qid)):
        counter += 1

1
2
3
5
6
7
8
9
10
11
12
13
14
<Response [429]>
16
<Response [429]>
18
<Response [429]>
20
<Response [429]>
22
<Response [429]>
24
<Response [429]>
26
<Response [429]>
28
<Response [429]>
30
<Response [429]>
32
<Response [429]>
34
<Response [429]>
36
<Response [429]>
38
<Response [429]>
40
<Response [429]>
42
<Response [429]>
44
<Response [429]>
46
<Response [429]>
48
<Response [429]>
50
51
<Response [429]>
53
<Response [429]>
55
<Response [429]>
57
<Response [429]>
59
<Response [429]>
61
<Response [429]>
63
<Response [429]>
65
<Response [429]>
67
68
<Response [429]>
70
<Response [429]>
72
<Response [429]>
74
<Response [429]>
76
<Response [429]>
78
<Response [429]>
80
<Response [429]>
82
<Response [429]>
84
<Response [429]>
86
<Response [429]>
88
<Response [429]>
90
<Response [429]>
92
<Response [429]>
94
<Response [429]>
96
97
<Response [429]>
99
<Response [429]>
101
<Response [429]>
103
<Response [429]>
105
<Response [429]>
107
<Response [429]>
109
<Response [429]>
111
<Response [429

KeyboardInterrupt: 

In [10]:
# counter

In [11]:
start = time.time()
relation_id_name = parallal_task_2(find_relation_names_and_ids, qids[:100])
print(time.time() - start)
len(relation_id_name)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Process ForkPoolWorker-9:
Process ForkPoolWorker-2:
Process ForkPoolWorker-12:
Process ForkPoolWorker-8:
Process ForkPoolWorker-10:
Process ForkPoolWorker-1:
Process ForkPoolWorker-3:
Process ForkPoolWorker-4:
Process ForkPoolWorker-7:
Process ForkPoolWorker-5:
Process ForkPoolWorker-6:
Process ForkPoolWorker-11:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/HadyElkady/miniconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/HadyElkady/miniconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/HadyElkady/miniconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootst

KeyboardInterrupt
  File "/home/HadyElkady/miniconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/home/HadyElkady/miniconda3/lib/python3.7/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
KeyboardInterrupt
  File "/home/HadyElkady/miniconda3/lib/python3.7/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
Process ForkPoolWorker-14:
Process ForkPoolWorker-13:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/HadyElkady/miniconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/HadyElkady/miniconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/HadyElkady/mi

In [ ]:
relation_id_name

In [ ]:
ex = {'significant event': 1}
outpput = find_relation_names_and_ids('Q90')
dict(ex, **outpput)

In [ ]:
total_len = 0
for r in relations[:90]:
    if len(r) < 40:
        total_len += len(r)
total_len

In [ ]:
qids

In [ ]:
len(target_relation)

In [ ]:
get_relation_label('P272')

In [ ]:
start = time.time()
relation_id_name = parallal_task(get_relation_label, target_relation)
print(time.time() - start)
len(relation_id_name)

In [ ]:
relation_id_name

In [ ]:
df = pd.DataFrame(relation_id_name, columns=['id', 'name'])
df['length'] = df["name"].apply(lambda x:len(x))
df

In [ ]:
len(df)

In [ ]:
df['length'].hist(bins=10)

In [ ]:
df['length'].hist(bins=20)

In [ ]:
len(df[df['length'] > 35])*100/len(df)

In [ ]:
len(df[df['length'] > 40])*100/len(df)

In [ ]:
len('category for people who died here')